In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

100%|██████████| 23040/23040 [00:00<00:00, 38425.83it/s]


In [3]:
COLUMNS = [
    # "facts",
    "features",
    "optimiser",
    "iterations",
    "pruning",
]

In [4]:
def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.pruning == "none"]
    # df = df[df.iterations == "4"]
    df = df[df.facts == "fd"]
    return df

for metric in [
    # "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    df = df.fillna(0)
    display(df)

solved


domain                                          blocksworld  childsnack  \
features optimiser iterations pruning                                     
lwl2     rank-svm  2          collapse-all               11           9   
                              collapse-layer             11           8   
                              collapse-layer-x           11           9   
                              none                        0           8   
                   4          collapse-all                0           7   
                              collapse-layer              9           7   
                              collapse-layer-x           11           7   
                              none                        0           6   
                   6          collapse-all                0           7   
                              collapse-layer             11           6   
                              collapse-layer-x           11           7   
                              none                        0           7   
         svr       2          collapse-all                8           5   
                              collapse-layer              8           4   
                              collapse-layer-x            8           5   
                              none                        9           3   
                   4          collapse-all                0           4   
                              collapse-layer              9           4   
                              collapse-layer-x            8           5   
                              none                        0           3   
                   6          collapse-all                0           5   
                              collapse-layer              8           4   
                              collapse-layer-x            8           5   
                              none                        0           3   
wl       rank-svm  2          collapse-all               27           7   
                              collapse-layer             26          10   
                              collapse-layer-x           16          10   
                              none                       26           8   
                   4          collapse-all               23          12   
                              collapse-layer             23          12   
                              collapse-layer-x           15          10   
                              none                       21          10   
                   6          collapse-all               22          12   
                              collapse-layer             20          10   
                              collapse-layer-x           16          10   
                              none                        0          11   
         svr       2          collapse-all               12           6   
                              collapse-layer             11           6   
                              collapse-layer-x           16           5   
                              none                       11           6   
                   4          collapse-all               18           4   
                              collapse-layer             11           5   
                              collapse-layer-x           16           6   
                              none                       21           6   
                   6          collapse-all               18           4   
                              collapse-layer              9           6   
                              collapse-layer-x           16           6   
                              none                       19           4   

domain                                          ferry  miconic  rovers  \
features optimiser iterations pruning                                    
lwl2     rank-svm  2          collapse-all         18       20       0   
                           

In [5]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"] + " - " + df["pruning"]
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            symbol="pruning",
            line_dash="pruning",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)